In [17]:
import sqlite3
import pandas as pd

class feedzaiChallenge():
    def __init__(self,database_name:str):
        self.database_name = database_name
        self.conn = sqlite3.connect(f'database\\{self.database_name}.db')

    def read_csv_files(self, csv_files:dict):
        try:
            self.dfs = dict()
            for file in csv_files:
                self.dfs[file] = pd.read_csv(csv_files[file])
        except Exception as ex:
            raise ex
    
    def load_data(self, table_name:str):
        try:
            self.dfs[table_name].to_sql(table_name, self.conn, if_exists='replace', index=False)
        except Exception as ex:
            raise ex
    
    def query_data(self,query, output_path:str):
        try:           
            result = pd.read_sql_query(query, self.conn)
            result.to_csv(output_path, index=False)
        except Exception as ex:
            raise ex
        
    def close_connection(self):
        self.conn.close()


In [18]:
feedzai = feedzaiChallenge('feedzai_database')

In [19]:
csv_files = {
    'time_off':r'csv_sources\time_off.csv',
    'work_hours':r'csv_sources\work_hours.csv'
}

In [20]:
feedzai.read_csv_files(csv_files)

In [21]:
feedzai.load_data('work_hours')
feedzai.load_data('time_off')

In [22]:
query = f"""
WITH daily_accumulated_cost AS (
    SELECT
        project_id,
        date,
        (SUM(worked) / 3600) * 100 AS accumulated_cost
    FROM
        work_hours
    GROUP BY
        project_id, date
    ORDER BY
        project_id, date
)
SELECT
    project_id,
    date,
    SUM(accumulated_cost) OVER (PARTITION BY project_id ORDER BY date) AS total_accumulated_cost
FROM
    daily_accumulated_cost;
"""
feedzai.query_data(query,r'output_files\acumulated_actual_costs.csv')


In [23]:
feedzai.close_connection()

# Query 1

In [ ]:
WITH date_bounds AS (
    SELECT
        MIN(date) AS min_date,
        MAX(date) AS max_date
    FROM work_hours
), weekdays AS (
    SELECT min_date AS work_date
    FROM date_bounds
    UNION ALL
    SELECT date(work_date, '+1 day')
    FROM weekdays, date_bounds
    WHERE work_date < max_date
), workdays AS (
    SELECT work_date
    FROM weekdays
    WHERE strftime('%w', work_date) NOT IN ('0', '6')  -- Exclude weekends
), employee_workdays AS (
    SELECT
        e.employee_id,
        e.employee_name,
        w.work_date,
        8 * 3600 AS available_seconds  -- 8 hours per day in seconds
    FROM
        (SELECT DISTINCT employee_id, employee_name FROM work_hours) e
    CROSS JOIN workdays w
    LEFT JOIN time_off t ON e.employee_id = t.employee_id AND w.work_date BETWEEN t.date_start AND t.date_end
    WHERE t.employee_id IS NULL
)
, total_worked_hours AS (
    SELECT
        employee_id,
        project_id,
        strftime('%Y-%m', date) AS month,
        SUM(worked) AS total_worked_seconds
    FROM work_hours
    GROUP BY employee_id, project_id, strftime('%Y-%m', date)
), total_available_hours AS (
    SELECT
        employee_id,
        strftime('%Y-%m', work_date) AS month,
        SUM(available_seconds) AS total_available_seconds
    FROM employee_workdays
    GROUP BY employee_id, strftime('%Y-%m', work_date)
)
SELECT
    a.employee_id,
    w.project_id,
    a.month,
    a.total_available_seconds,
    COALESCE(w.total_worked_seconds, 0) AS total_worked_seconds,
    ROUND((COALESCE(w.total_worked_seconds, 0) * 1.0 / a.total_available_seconds) * 100, 2) AS utilization_percentage
FROM
    total_available_hours a
LEFT JOIN total_worked_hours w ON a.employee_id = w.employee_id AND a.month = w.month
ORDER BY
    a.employee_id, w.project_id, a.month;


# Query 2

In [ ]:
WITH date_bounds AS (
    SELECT
        MIN(date) AS min_date,
        MAX(date) AS max_date
    FROM work_hours
),
calendar AS (
    WITH RECURSIVE dates(date) AS (
        SELECT min_date AS date
        FROM date_bounds
        UNION ALL
        SELECT date(date, '+1 day')
        FROM dates
        WHERE date < (SELECT max_date FROM date_bounds)
    )
    SELECT date
    FROM dates
),
work_days AS (
    SELECT 
        w.employee_id, 
        w.employee_name, 
        strftime('%Y-%m', c.date) AS month, 
        COUNT(*) AS working_days
    FROM calendar c
    JOIN work_hours w ON strftime('%Y-%m', c.date) = strftime('%Y-%m', w.date)
    LEFT JOIN time_off t ON c.date BETWEEN t.date_start AND t.date_end AND w.employee_id = t.employee_id
    WHERE strftime('%w', c.date) NOT IN ('0', '6') -- exclude weekends
    AND t.employee_id IS NULL -- exclude time off
    GROUP BY w.employee_id, month
),
project_hours AS (
    SELECT 
        employee_id, 
        strftime('%Y-%m', date) AS month, 
        project_id, 
        SUM(worked / 3600.0) AS total_hours  -- divide by 3600.0 to get hours
    FROM work_hours
    GROUP BY employee_id, month, project_id
)
SELECT 
    p.employee_id, 
    p.month, 
    p.project_id, 
    ROUND((p.total_hours / (w.working_days * 8)) * 100, 2) AS utilization_percentage
FROM project_hours p
JOIN work_days w ON p.employee_id = w.employee_id AND p.month = w.month;
